In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.models import load_model

In [2]:
def loadDataFiles():
    market_df = pickle.load(open('Market_train',"rb"))
    news_df = pickle.load(open("News_train", "rb"))
    print('Finished loading datafiles!')
    return market_df, news_df


In [3]:
def preprocess_data(mkt_df, news_df):
    mkt_df['time'] = pd.to_datetime(mkt_df['time'])
    news_df['time'] = pd.to_datetime(news_df['time'])
    mkt_df['time'] = mkt_df['time'].dt.date
    news_df['time'] = news_df['time'].dt.date
    assetCodes = []
    index = 0
    for x in news_df['assetCodes']:
        x = x.split(',')[0].split("'")[1]
        assetCodes.append(x)
    news_df['assetCode'] = np.asarray(assetCodes)
    irrelevantColumns = ['sourceTimestamp', 'firstCreated', 'sourceId', 
                         'headline', 'provider', 'subjects', 'audiences',
                        'headlineTag', 'marketCommentary', 'assetCodes', 'assetName']
    news_df.drop(irrelevantColumns, axis=1, inplace=True)
    mkt_df.drop(['assetName'], axis=1, inplace=True)
    modifiednews = news_df.groupby(['time','assetCode'], sort=False).aggregate(np.mean).reset_index()
    
    # join news reports to market data, note many assets will have many days without news data
    merged = pd.merge(mkt_df, modifiednews, how='left', on=['time', 'assetCode'], copy=False) 
    merged = merged.fillna(0)
    print('Finished preprocessing data!')
    return merged


In [4]:
market_data, news_data = loadDataFiles()


Finished loading datafiles!


In [5]:
X = preprocess_data(market_data, news_data)


Finished preprocessing data!


In [6]:
def normalizeY(ydf):
    ydf = (ydf + 1) / 2
    return ydf


In [7]:
X = X[X['returnsOpenNextMktres10'] >= -1]
X = X[X['returnsOpenNextMktres10'] <= 1]

y = X['returnsOpenNextMktres10']

X.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)
y = normalizeY(y)
assetCodesAndTime = X.iloc[:, :2]
X = X.iloc[:, 2:]


In [8]:
def getLinearRegressionModel(numfeatures):
    inputs = keras.layers.Input(shape=(numfeatures,))
    preds = keras.layers.Dense(1,activation='linear')(inputs)
    model = keras.Model(inputs=inputs,outputs=preds)
    sgd=keras.optimizers.SGD(.1)
    model.compile(optimizer=sgd ,loss='mean_absolute_error',metrics=['mae'])
    return model


In [9]:
def regularize(df):
    for column in df:
        colmin = np.amin(df[column])
        colmax = np.amax(df[column])
        df[column] = (df[column] - colmin) / (colmax - colmin)
    return df

In [10]:
X = regularize(X)

In [11]:
def saveModel(model, model_name):
    model.save(model_name + '.h5')

In [12]:
def loadModel(filename):
    model = load_model(filename)
    return model

In [13]:
def modify_dataset(X, degree):
    if degree == 0 or degree == 1:
        return X
    for x in range(degree - 1):
        for column in X.columns.values:
            newcol = column + '_' + str((x+2))
            X[newcol] = X[column] ** (x + 2)
    return X

In [14]:
degree = 3
X = modify_dataset(X, degree)
lrmodel = getLinearRegressionModel(len(X.columns.values))
lrmodel.fit(X,y, batch_size=1000000, epochs=10, validation_split=.3)

Train on 2850542 samples, validate on 1221661 samples
Epoch 1/10
2850542/2850542 [==============================] - 35s 12us/step - loss: 0.4111 - mean_absolute_error: 0.4111 - val_loss: 0.1509 - val_mean_absolute_error: 0.1509
Epoch 2/10
2850542/2850542 [==============================] - 30s 10us/step - loss: 0.1455 - mean_absolute_error: 0.1455 - val_loss: 0.1728 - val_mean_absolute_error: 0.1728
Epoch 3/10
2850542/2850542 [==============================] - 28s 10us/step - loss: 0.1542 - mean_absolute_error: 0.1542 - val_loss: 0.1213 - val_mean_absolute_error: 0.1213
Epoch 4/10
2850542/2850542 [==============================] - 26s 9us/step - loss: 0.1459 - mean_absolute_error: 0.1459 - val_loss: 0.1857 - val_mean_absolute_error: 0.1857
Epoch 5/10
2850542/2850542 [==============================] - 27s 9us/step - loss: 0.1647 - mean_absolute_error: 0.1647 - val_loss: 0.1287 - val_mean_absolute_error: 0.1287
Epoch 6/10
2850542/2850542 [==============================] - 28s 10us/step - 